In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [27]:
URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=15174;type=tournament'
page = requests.get(URL)
page

<Response [200]>

# Getting Matchwise Information of WPL 2023

In [28]:
bs = BeautifulSoup(page.content,"html.parser")
table_body=bs.find_all('tbody')
table_body

[<tbody>
 <tr class="data1" data-days="738948">
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/1358722.html">GG Women</a></td>
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/1358720.html">MI Women</a></td>
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/1358720.html">MI Women</a></td>
 <td class="left" nowrap="nowrap">143 runs</td>
 <td class="left"><a class="data-link" href="/ci/content/ground/343050.html">DY Patil</a></td>
 <td nowrap="nowrap">Mar 4, 2023</td>
 <td nowrap="nowrap"><a class="data-link" href="/ci/engine/match/1358929.html">WT20</a></td>
 </tr>
 <tr class="data1" data-days="738949">
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/1358723.html">RCB Women</a></td>
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/1358721.html">DC Women</a></td>
 <td class="left" nowrap="nowrap"><a class="data-link" href="/ci/content/team/135

In [29]:
matchSummary_df = pd.DataFrame(columns=["team1","team2","winner", "margin", "ground", "matchDate", "scorecard"])

In [30]:
for i, table in enumerate(table_body):
        rows = table.find_all('tr')
        for j,row in enumerate(rows):
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            matchSummary_df = matchSummary_df.append(pd.Series([
            cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], 'WPL #' + str(j+1)], 
            index=matchSummary_df.columns ), ignore_index=True)

C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\4245687790.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matchSummary_df = matchSummary_df.append(pd.Series([


In [31]:
matchSummary_df

,team1,team2,winner,margin,ground,matchDate,scorecard
0,GG Women,MI Women,MI Women,143 runs,DY Patil,"Mar 4, 2023",WPL #1
1,RCB Women,DC Women,DC Women,60 runs,Brabourne,"Mar 5, 2023",WPL #2
2,UPW Women,GG Women,UPW Women,3 wickets,DY Patil,"Mar 5, 2023",WPL #3
3,MI Women,RCB Women,MI Women,9 wickets,Brabourne,"Mar 6, 2023",WPL #4
4,DC Women,UPW Women,DC Women,42 runs,DY Patil,"Mar 7, 2023",WPL #5
5,GG Women,RCB Women,GG Women,11 runs,Brabourne,"Mar 8, 2023",WPL #6
6,DC Women,MI Women,MI Women,8 wickets,DY Patil,"Mar 9, 2023",WPL #7
7,RCB Women,UPW Women,UPW Women,10 wickets,Brabourne,"Mar 10, 2023",WPL #8
8,GG Women,DC Women,DC Women,10 wickets,DY Patil,"Mar 11, 2023",WPL #9
9,UPW Women,MI Women,MI Women,8 wickets,Brabourne,"Mar 12, 2023",WPL #10


In [32]:
matchSummary_df.to_csv('RawData/wpl_matchSummary_2k23.csv')
matchSummary_df.to_json('RawData/wpl_matchSummary_2k23.json')

# Getting Batting Summary 

In [33]:
matchlinks = []
for i, table in enumerate(table_body):
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            rowURL = "https://www.espncricinfo.com" + str(cols[6].find('a')['href'])
            matchlinks.append(rowURL)

In [34]:
battingSummary_df = pd.DataFrame(columns=["match","teamInnings","battingPos", "batsmanName", "dismissal", "runs", "balls","4s","6s","SR"])
for link in matchlinks:
    URL = link
    page1 = requests.get(URL)
    bs1 = BeautifulSoup(page1.content,"html.parser")
    teamInfo = bs1.find(class_='ds-bg-fill-canvas').get_text().split(' Innings')
    scorecards = bs1.find_all(class_='ci-scorecard-table') 
    team1 = teamInfo[0]
    team2 = teamInfo[1]
    matchInfo = team1 +  ' Vs ' + team2
    
    FIRows = scorecards[0].tbody.find_all('tr')
    SERows = scorecards[1].tbody.find_all('tr')
    firstInningRows = [row for row in FIRows if len(row.find_all('td')) >= 8]
    secondInningRows = [row for row in SERows if len(row.find_all('td')) >= 8] 
            
    for index,bat1Row in enumerate(firstInningRows):
        tds = bat1Row.find_all('td')
        battingSummary_df = battingSummary_df.append(pd.Series([
                matchInfo,team1,index+1,tds[0].get_text(),tds[1].get_text(),tds[2].get_text(),
                tds[3].get_text(),tds[5].get_text(),tds[6].get_text(),tds[7].get_text()], 
                index=battingSummary_df.columns), ignore_index=True)

    for index1,bat2Row in enumerate(secondInningRows):
        tds = bat2Row.find_all('td')
        battingSummary_df = battingSummary_df.append(pd.Series([
                matchInfo,team2,index1+1,tds[0].get_text(),tds[1].get_text(),tds[2].get_text(),
                tds[3].get_text(),tds[5].get_text(),tds[6].get_text(),tds[7].get_text()], 
                index=battingSummary_df.columns), ignore_index=True)       
    

C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\3993661825.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  battingSummary_df = battingSummary_df.append(pd.Series([
C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\3993661825.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  battingSummary_df = battingSummary_df.append(pd.Series([


In [35]:
battingSummary_df

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,MI Women Vs GG Women,MI Women,1,Yastika Bhatia †,c Wareham b Kanwar,1,8,0,0,12.50
1,MI Women Vs GG Women,MI Women,2,Hayley Matthews,b Gardner,47,31,3,4,151.61
2,MI Women Vs GG Women,MI Women,3,Nat Sciver-Brunt,c Rana b Wareham,23,18,5,0,127.77
3,MI Women Vs GG Women,MI Women,4,Harmanpreet Kaur (c),c Hemalatha b Rana,65,30,14,0,216.66
4,MI Women Vs GG Women,MI Women,5,Amelia Kerr,not out,45,24,6,1,187.50
...,...,...,...,...,...,...,...,...,...,...
331,DC Women Vs MI Women,MI Women,1,Hayley Matthews,c Reddy b Jonassen,13,12,3,0,108.33
332,DC Women Vs MI Women,MI Women,2,Yastika Bhatia †,c Capsey b Yadav,4,3,1,0,133.33
333,DC Women Vs MI Women,MI Women,3,Nat Sciver-Brunt,not out,60,55,7,0,109.09
334,DC Women Vs MI Women,MI Women,4,Harmanpreet Kaur (c),run out (Pandey/Capsey),37,39,5,0,94.87


In [36]:
battingSummary_df.to_csv('RawData/wpl_battingSummary_2k23.csv')
battingSummary_df.to_json('RawData/wpl_battingSummary_2k23.json')

# Getting Bowling Summary 


In [37]:
bowlingSummary_df = pd.DataFrame(columns=["match","bowlingTeam","bowlerName", "overs", "maiden", "runs", "wickets","economy","0s","4s","6s","wides","noBalls"])
for link in matchlinks:
    URL = link
    page2 = requests.get(URL)
    bs2 = BeautifulSoup(page2.content,"html.parser")
    teamInfo = bs2.find(class_='ds-bg-fill-canvas').get_text().split(' Innings')
    scorecards = bs2.find_all(class_='ds-table') 
    team1 = teamInfo[0]
    team2 = teamInfo[1]
    matchInfo = team1 +  ' Vs ' + team2
    
    FIRows = scorecards[1].tbody.find_all('tr')
    SERows = scorecards[3].tbody.find_all('tr')
    
    firstInningRows = [row for row in FIRows if len(row.find_all('td')) >= 11]
    secondInningRows = [row for row in SERows if len(row.find_all('td')) >= 11] 
            
    for index,ball1Row in enumerate(firstInningRows):
        tds = ball1Row.find_all('td')
        bowlingSummary_df = bowlingSummary_df.append(pd.Series([
                matchInfo,team1,tds[0].get_text(),tds[1].get_text(),tds[2].get_text(),tds[3].get_text(),tds[4].get_text(),
                tds[5].get_text(),tds[6].get_text(),tds[7].get_text(),tds[8].get_text(),tds[9].get_text(),tds[10].get_text()], 
                index=bowlingSummary_df.columns), ignore_index=True)
    
    for index,ball2Row in enumerate(secondInningRows):
        tds = ball2Row.find_all('td')
        bowlingSummary_df = bowlingSummary_df.append(pd.Series([
                matchInfo,team2,tds[0].get_text(),tds[1].get_text(),tds[2].get_text(),tds[3].get_text(),tds[4].get_text(),
                tds[5].get_text(),tds[6].get_text(),tds[7].get_text(),tds[8].get_text(),tds[9].get_text(),tds[10].get_text()], 
                index=bowlingSummary_df.columns), ignore_index=True)

C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\1441929970.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bowlingSummary_df = bowlingSummary_df.append(pd.Series([
C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\1441929970.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bowlingSummary_df = bowlingSummary_df.append(pd.Series([


In [38]:
bowlingSummary_df

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,MI Women Vs GG Women,MI Women,Ashleigh Gardner,4,0,38,1,9.50,7,7,0,0,0
1,MI Women Vs GG Women,MI Women,Mansi Joshi,2,0,17,0,8.50,7,2,1,1,0
2,MI Women Vs GG Women,MI Women,Tanuja Kanwar,2,0,12,1,6.00,6,2,0,0,0
3,MI Women Vs GG Women,MI Women,Monica Patel,2,0,34,0,17.00,1,7,0,2,0
4,MI Women Vs GG Women,MI Women,Georgia Wareham,3,0,30,1,10.00,6,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,DC Women Vs MI Women,MI Women,Marizanne Kapp,4,0,22,0,5.50,12,3,0,1,0
266,DC Women Vs MI Women,MI Women,Radha Yadav,4,0,24,1,6.00,11,3,0,2,0
267,DC Women Vs MI Women,MI Women,Jess Jonassen,4,0,28,1,7.00,10,4,0,0,0
268,DC Women Vs MI Women,MI Women,Shikha Pandey,4,0,23,0,5.75,11,2,0,0,0


In [39]:
bowlingSummary_df.to_csv('RawData/wpl_bowlingSummary_2k23.csv')
bowlingSummary_df.to_json('RawData/wpl_bowlingSummary_2k23.json')

# Getting Player's Info


In [40]:
squadURL = 'https://www.espncricinfo.com/series/women-s-premier-league-2022-23-1348825/squads'
page3 = requests.get(squadURL)

In [41]:
squadLinks = []
bs3 = BeautifulSoup(page3.content,"html.parser")
squadList = bs3.find_all(class_='ds-mb-4')[0].find_all('a')
for links in squadList:
    teamURL = "https://www.espncricinfo.com" + str(links.get('href'))
    squadLinks.append(teamURL)

In [42]:
playerInfo_df = pd.DataFrame(columns=["playerName","team","battingStyle", "bowlingStyle", "playingRole"])
for link in squadLinks:
    URL = link
    page4 = requests.get(URL)
    bs4 = BeautifulSoup(page4.content,"html.parser")
    teamName = bs4.find(class_='ds-mb-4').span.get_text().replace('Squad',' ')
    sqaudPage = bs4.find(class_='lg:ds-grid-cols-2')
    for player in sqaudPage.children:
        playerName = str(player.find_all('a')[1].span.get_text())
        team = teamName
        battingStyle = player.find_all('p')[2].get_text().split(': ')[1]
        count = len(player.find_all('p'))
        bowlingStyle = None if len(player.find_all('p')) <= 3 else player.find_all('p')[3].get_text().split(': ')[1]
        playerImage = player.find('img')
        playingRole = player.find('p',class_='ds-text-tight-l').get_text()
        playerInfo_df = playerInfo_df.append(pd.Series([playerName,team,battingStyle,bowlingStyle,playingRole], 
        index=playerInfo_df.columns), ignore_index=True)

C:\Users\Kedhar\AppData\Local\Temp\ipykernel_10324\3180098040.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerInfo_df = playerInfo_df.append(pd.Series([playerName,team,battingStyle,bowlingStyle,playingRole],


In [43]:
playerInfo_df

,playerName,team,battingStyle,bowlingStyle,playingRole
0,Meg Lanning,Delhi Capitals Women,Right hand Bat,Right arm Medium,Top order Batter
1,Taniya Bhatia,Delhi Capitals Women,Right hand Bat,None,Wicketkeeper Batter
2,Alice Capsey,Delhi Capitals Women,Right hand Bat,Right arm Offbreak,Batting Allrounder
3,Laura Harris,Delhi Capitals Women,Right hand Bat,None,Top order Batter
4,Jasia Akhtar,Delhi Capitals Women,Right hand Bat,Right arm Medium,
...,...,...,...,...,...
85,Shivali Shinde,UP Warriorz Women,Right hand Bat,None,
86,Simran Shaikh,UP Warriorz Women,Right hand Bat,Legbreak,Middle order Batter
87,Devika Vaidya,UP Warriorz Women,Left hand Bat,Legbreak Googly,Allrounder
88,Soppadhandi Yashasri,UP Warriorz Women,Right hand Bat,Right arm Medium fast,


In [44]:
playerInfo_df.to_csv('RawData/wpl_playerInfo_df_2k23.csv')
playerInfo_df.to_json('RawData/wpl_playerInfo_df_2k23.json')